In [1]:
import urllib.request
import requests
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 800)

import functools
import datetime

## Request Prostitution-related Data from the WPRDC API

In [2]:
# Base URL suggested by WPRDC
# url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=e03a89dd-134a-4ee8-a2bd-62c40aeebc6f&limit=5&q=title:jones'  

In [62]:
req_limit = 32000 # number of records to pull
test_url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=e03a89dd-134a-4ee8-a2bd-62c40aeebc6f&q={"OFFENSES":"5902"}&limit='+str(req_limit)

In [63]:
r = requests.get(test_url).json()
r['success']

True

In [82]:
r

{'help': 'https://data.wprdc.org/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'limit': 32000,
  'q': {'OFFENSES': '5902'},
  'records_format': 'objects',
  'resource_id': 'e03a89dd-134a-4ee8-a2bd-62c40aeebc6f',
  'total_estimation_threshold': None,
  'records': [{'_id': 73054,
    'PK': '2047953',
    'CCR': '22063258',
    'AGE': 39,
    'GENDER': 'M',
    'RACE': 'B',
    'ARRESTTIME': '2022-05-18T11:30:00',
    'ARRESTLOCATION': 'Zone 1',
    'OFFENSES': '3304 Criminal Mischief. / 5902(a)(1) Prostitution  / 3124.1 Sexual Assault / 5902(b) Promoting Prostitution / 5902(b)(1) Promoting Prostitution - Keeping Business or House / 5902(b)(3) Promoting Prostitution - Causing another to become or remain  / 5902(b)(7) Permitting place to be regularly used for prostitution / 5902(b)(8) Prostitution - receiving benefit from / 3121(a)(3) Rape of unconscious person / 3123(a)(3) IDSI with unconscious person / 3126(a)(4) Indecent assault of

## Format arrest data

In [83]:
data = pd.DataFrame(r['result']['records'])
data = data.set_index('_id')
data.index.rename('id', inplace=True)
data.sample(5)

,PK,CCR,AGE,GENDER,RACE,ARRESTTIME,ARRESTLOCATION,OFFENSES,INCIDENTLOCATION,INCIDENTNEIGHBORHOOD,INCIDENTZONE,INCIDENTTRACT,COUNCIL_DISTRICT,PUBLIC_WORKS_DIVISION,X,Y,rank OFFENSES
id,,,,,,,,,,,,,,,,,
5026,1980873,17016221,25.0,F,B,2017-01-26T22:18:00,"Bigelow SQ Pittsburgh, PA 15219",5902(a)(1) Prostitution,"Bigelow SQ Pittsburgh, PA 15219",Central Business District,2,201,6,6,-79.994404,40.440786,0.057309
36442,2016386,19139626,64.0,M,B,2019-07-15T21:32:00,"Middle ST & Foreland ST Pittsburgh, PA 15212",902 Criminal Solicitation. / 5902(e) Patronizing a Prostitute,"Middle ST & Foreland ST Pittsburgh, PA 15212",East Allegheny,1,2304,1,1,-79.999393,40.454577,0.057309
50756,2028532,20001420,34.0,F,B,2020-07-19T08:41:00,"1400 Block Washington BL Pittsburgh, PA 15206",5902(a)(1) Prostitution / 5902(a)(2) Prostitution - Loitering for purpose of,"E North AV & Howard ST Pittsburgh, PA 15212",East Allegheny,1,2304,1,1,-79.999236,40.456847,0.070624
35962,2015938,19129794,37.0,M,B,2019-07-02T19:21:00,"100 Block Washington PL Pittsburgh, PA 15219",907 Possessing Instruments of Crime. / 5902(a)(1) Prostitution / 13(a)(16) Possession of Controlled Substance / 13(a)(16) Possession of Controlled Substance / 13(a)(30) Possession w/ Intent to Del. Controlled Substance / 13(a)(30) Possession w/ Intent to Del. Controlled Substance / 13(a)(31) Marijuana: Possession Small Amount,"100 Block Washington PL Pittsburgh, PA 15219",Central Business District,2,201,9,2,-79.906161,40.464050,0.057309
23257,2004035,18147696,50.0,M,W,2018-08-03T11:06:00,"600 Block 1st AV Pittsburgh, PA 15219",5902(e) Patronizing a Prostitute,"East ST & E North AV Pittsburgh, PA 15212",East Allegheny,1,2304,1,1,-79.998831,40.456928,0.057309


## Categorize prostitution-related charges

In [84]:
charges_dict = {
    "providers" : {
        "5902\s?\(a\)": 'Prostitution',
        "5902\s?\(a\)\(1\)": 'Prostitution',
        "5902\s?\(a\)\(2\)": 'Prostitution - Loitering for',
    },
    "third-party" : {
        "5902\s?\(b\)": 'Promoting',
        "5902\s?\(b\)\(1\)": 'Promoting - Keeping house or business for ',
        "5902\s?\(b\)\(2\)": 'Promoting - Finding brothelworker',
        "5902\s?\(b\)\(3\)": 'Promoting - Encouraging',
        "5902\s?\(b\)\(4\)": 'Promoting - Soliciting patron for',
        "5902\s?\(b\)\(5\)": 'Promoting - Procuring prostitute for patron',
        "5902\s?\(b\)\(6\)": 'Promoting - Transporting for',
        "5902\s?\(b\)\(7\)": 'Promoting - Leasing or regular use of place',
        "5902\s?\(b\)\(8\)": 'Promoting - Receiving benefit',
        "5902\s?\(d\)": 'Living off prostitutes',
    },
    "clients": {   
        "5902\s?\(e\)": 'Patronizing prostitutes',
    }
}

charges_by_p = {
    "providers" : ["5902(a)", "5902(a)(1)", "5902(a)(2)"],
    "third-party" : ["5902(b)(1)", "5902(b)(2)", "5902(b)(3)", "5902(b)(4)", "5902(b)(5)", "5902(b)(6)", "5902(b)(7)", "5902(b)(8)", "5902(d)"], 
    "clients" : ["5902(e)"]
}

In [85]:
## Add boolean column for each type of charge
provider_mask = data.OFFENSES.str.contains('|'.join(charges_dict["providers"]))
third_mask = data.OFFENSES.str.contains('|'.join(charges_dict["third-party"]))
client_mask = data.OFFENSES.str.contains('|'.join(charges_dict["clients"]))

data["ISPROVIDER"] = provider_mask
data["ISTHIRDPARTY"] = third_mask
data["ISCLIENT"] = client_mask

In [86]:
# Examine overlap in types of charges
data[["ISTHIRDPARTY", "ISPROVIDER", "ISCLIENT"]].value_counts()

ISTHIRDPARTY  ISPROVIDER  ISCLIENT
False         True        False       460
              False       True        176
True          False       False        53
              True        False        16
False         True        True          2
Name: count, dtype: int64

In [87]:
data.AGE.describe()

count    700.000000
mean      37.394286
std       12.173706
min        0.000000
25%       28.000000
50%       35.000000
75%       44.000000
max       88.000000
Name: AGE, dtype: float64

## Convert data types, reformat

In [88]:
## Convert date format
data.ARRESTTIME = pd.to_datetime(data.ARRESTTIME)

## Add year & month column for convenience
data["YEAR"] = data.ARRESTTIME.dt.year
data["MONTH"] = data.ARRESTTIME.dt.month

## Covnert offense string to list
data.OFFENSES = data.OFFENSES.apply(lambda offs: [off.strip() for off in offs.split(" / ")])

In [89]:
## Fill missing values with NaN 
data.AGE = data.AGE.replace(0.0, np.nan)


In [90]:
base_data = data.copy()

# About this data

This data comes from the Western Pennsylvania Regional Data Center. It represents arrests made by or within the *city of Pittsburgh*. This data does not usually represent arrests made by Allegheny County police or by police in the surrounding boroughs. 

### Q: How many prostitution-related arrests were made?

A: There are 707 (and counting) arrests available in this dataset. We include any charges under 5902, the PA state statute that includes charges related to prostitution (providing sex work), being a patron (a john/customer), or promoting prosititution (assisting in prostitution, pimping, running a brothel, etc). These charges are not focused on sex trafficking, sexual assault, or coercion.   

In [91]:
len(base_data)

707

# Questions - people arrested for prostitution

### Q: How many people were arrested for Prostitution from 2013-today?

Note: It seems like sometimes police tack on a prostitution charge (5902(a)) for any charges related to prostitution/soliciting/promotion. 

In [92]:
start_date = "2013-01-01"
#end_date = "2019-01-01"
end_date = pd.Timestamp.today()

data = data.loc[(data.ARRESTTIME > start_date) & (data.ARRESTTIME < end_date)]
output = data.loc[data.ISPROVIDER][["YEAR"]]
pd.DataFrame(output.value_counts(sort=False))

,count
YEAR,
2016,45
2017,106
2018,105
2019,109
2020,64
2021,21
2022,21
2023,7


### Q: What are the demographics of those arrested for prostitution?

In [93]:
providers = data.loc[data.ISPROVIDER]
providers.AGE.value_counts(bins=6, sort=False, dropna=False)

AGE
(16.927999999999997, 28.833]    140
(28.833, 40.667]                215
(40.667, 52.5]                   86
(52.5, 64.333]                   27
(64.333, 76.167]                  2
(76.167, 88.0]                    1
Name: count, dtype: int64

In [94]:
providers.AGE.describe()

count    471.000000
mean      34.895966
std       10.076717
min       17.000000
25%       27.500000
50%       34.000000
75%       40.000000
max       88.000000
Name: AGE, dtype: float64

In [95]:
## Under 18?
providers.loc[providers.AGE < 18]

,PK,CCR,AGE,GENDER,RACE,ARRESTTIME,ARRESTLOCATION,OFFENSES,INCIDENTLOCATION,INCIDENTNEIGHBORHOOD,...,COUNCIL_DISTRICT,PUBLIC_WORKS_DIVISION,X,Y,rank OFFENSES,ISPROVIDER,ISTHIRDPARTY,ISCLIENT,YEAR,MONTH
id,,,,,,,,,,,,,,,,,,,,,
7617,1984102,17071348,17.0,F,B,2017-04-20 17:20:00,"Moravian ST & Foreland ST Pittsburgh, PA 15212","[5902(a)(1) Prostitution, 5902(a)(2) Prostitution - Loitering for purpose of]","Moravian ST & Foreland ST Pittsburgh, PA 15212",East Allegheny,...,1,1,-80.001390,40.454185,0.070624,True,False,False,2017,4
18605,1998124,18047942,17.0,F,W,2018-03-14 18:21:00,"600 Block Commonwealth PL Pittsburgh, PA 15222",[5902(a)(1) Prostitution],"600 Block Commonwealth PL Pittsburgh, PA 15222",Central Business District,...,6,6,-80.006405,40.441733,0.057309,True,False,False,2018,3


# Charges Associated with Prostitution

### Q: What charges are commonly associated with prostitution? 
A: Instruments of crime, criminal solicitation, drug charges, and criminal conspiracy are common cross-charges. 

In [102]:
cutoff = 0 # ignore charges that show up very infrequently
data.OFFENSES.explode().value_counts().loc[lambda count : count > cutoff].tail(50)

OFFENSES
3503 Criminal Trespass                                                                             2
9126 Prostitution Investigation                                                                    2
4703 Operation of Vehicle without Official Certif. of Inspection                                   2
5503 Disorderly Conduct.                                                                           2
7513 Restriction on Alcoholic Beverages - Open Container                                           2
3334 Turning Movements and Required Signals.                                                       2
3125 Aggravated Indecent Assault.                                                                  2
2709(a)(1) Harassment by Physical Contact, or Attempts or Threats                                  2
2701(a)(1) Simple Assault - Intent., Know., Reckless.Cause Bod. Injury                             2
2718(a)(1) Strangulation - Applying Pressure to the Throat or Neck                

### Q: How many charges are usually placed?
Most people with a prostitution charge (5902(a)(1)) also pick up 1-2 additional charges. Loitering for the purpose of prostitution is the most common accompanying charge (5902(a)(2)), followed by possession of an instrument of crime. 

According to a 2018 TribLive article, multiple charges might be used as justification to escalate charges and book people in to the station: 
> The prostitution charge against Sabatini, then 39, was a summary offense. Had it been the only charge, the undercover detective who arrested Sabatini in Pittsburgh probably would have let the Buffalo, N.Y., native go the same night and mailed her a summons to appear in court. 
But the detective also filed a first-degree misdemeanor charge of possessing an instrument of crime in connection with the condom Sabatini spent several days in jail before negotiating a deal in which she pleaded guilty to the prostitution charge in exchange for getting the more serious, instrument-of-crime charge dropped

([TribLive source](https://archive.triblive.com/local/pittsburgh-allegheny/condoms-criminalized-in-allegheny-county-prostitution-cases/)). 

In [41]:
# Number of offenses charged
providers.OFFENSES.apply(len).describe().round(1)

count    428.0
mean       2.7
std        1.3
min        1.0
25%        2.0
50%        2.0
75%        3.0
max       12.0
Name: OFFENSES, dtype: float64

### Q: Are police still charging for "Instruments of crime" in prostitution?
A: Yes. The [Pittsburgh-area campaign](https://www.womenslawproject.org/2018/06/27/a-victory-of-sorts-allegheny-county-police-will-stop-criminalizing-condoms/) to stop criminalizing condoms was at its peak around 2018. Yet somehow, the use of this charge actually increased in 2019. We don't know what specific objects are being seized as the instrument of crime, but phones and condoms are common. 

In [42]:
## Are police still charging "Instrument of crime" alongisde prostitution?
charge = "907"
providers907 = providers[providers.OFFENSES.apply(lambda offs: any(charge in off for off in offs))]
providers907.YEAR.value_counts(sort=True)

YEAR
2019    40
2017    23
2020    19
2018    15
2016     4
2021     1
Name: count, dtype: int64

### Q: Did any of the charges involve minors?
A: TODO
There are many charges that might be related to the exploitation of minors. 
* 5902(b.1) Promoting prostitution of minor
* 6301 Corruption of Minors
* 6318 Unlawful Contact/Communication w/ Minor
* 2910 Luring a Child into a Motor Vehicle.* 
3011(b) Trafficking in Minos * 3131(c)(1) Unlawful dissemination of intimate image -- Person depicted IS a minor UNDER 18 (M1)
* 3126(a)(7) Indecent assault of person under 13 * 

4304(a)(1 "Endangering Welfare of Childre."At least two people have beenas arrested foprostitution at the age of 17. r 

In [104]:
""" TODO Make a filter of these charges and see how many cases total there are
5902(b.1) Promoting prostitution of minor
* 6301 Corruption of Minors
* 6318 Unlawful Contact/Communication w/ Minor
* 2910 Luring a Child into a Motor Vehicle.
* 3011(b) Trafficking in Minors
* 3131(c)(1) Unlawful dissemination of intimate image -- Person depicted IS a minor UNDER 18 (M1)
* 3126(a)(7) Indecent assault of person under 13
* 4304(a)(1) Endangering Welfare of Children.
"""


' TODO Make a filter of these charges and see how many cases total there are\n5902(b.1) Promoting prostitution of minor\n* 6301 Corruption of Minors\n* 6318 Unlawful Contact/Communication w/ Minor\n* 2910 Luring a Child into a Motor Vehicle.\n* 3011(b) Trafficking in Minors\n* 3131(c)(1) Unlawful dissemination of intimate image -- Person depicted IS a minor UNDER 18 (M1)\n* 3126(a)(7) Indecent assault of person under 13\n* 4304(a)(1) Endangering Welfare of Children.\n'

In [99]:
charge = "b.1"
data_minors = data[data.OFFENSES.apply(lambda offs: any(charge in off for off in offs))]
data_minors

,PK,CCR,AGE,GENDER,RACE,ARRESTTIME,ARRESTLOCATION,OFFENSES,INCIDENTLOCATION,INCIDENTNEIGHBORHOOD,...,COUNCIL_DISTRICT,PUBLIC_WORKS_DIVISION,X,Y,rank OFFENSES,ISPROVIDER,ISTHIRDPARTY,ISCLIENT,YEAR,MONTH
id,,,,,,,,,,,,,,,,,,,,,
75959,2050621,22124132,23.0,F,W,2022-08-04 07:48:00,"1600 Block 5th AV Pittsburgh, PA 15219","[902 Criminal Solicitation., 2706 Terroristic Threats., 5104 Resisting Arrest or Other Law Enforcement., 5503 Disorderly Conduct., 5505 Public Drunkenness, 3503(b.1) Criminal Trespass - Simple Trespasser, 5902(a)(2) Prostitution - Loitering for purpose of, 2709(a)(3) Harassment No Legitimate Purpose]","Miltenberger ST & 5th AV Pittsburgh, PA 15219",Bluff,...,6,3,-79.981369,40.438395,0.057309,True,False,False,2022,8


In [103]:
## People under the age of 18 arrested for prostitution
providers.loc[providers.AGE < 18]

,PK,CCR,AGE,GENDER,RACE,ARRESTTIME,ARRESTLOCATION,OFFENSES,INCIDENTLOCATION,INCIDENTNEIGHBORHOOD,...,COUNCIL_DISTRICT,PUBLIC_WORKS_DIVISION,X,Y,rank OFFENSES,ISPROVIDER,ISTHIRDPARTY,ISCLIENT,YEAR,MONTH
id,,,,,,,,,,,,,,,,,,,,,
7617,1984102,17071348,17.0,F,B,2017-04-20 17:20:00,"Moravian ST & Foreland ST Pittsburgh, PA 15212","[5902(a)(1) Prostitution, 5902(a)(2) Prostitution - Loitering for purpose of]","Moravian ST & Foreland ST Pittsburgh, PA 15212",East Allegheny,...,1,1,-80.001390,40.454185,0.070624,True,False,False,2017,4
18605,1998124,18047942,17.0,F,W,2018-03-14 18:21:00,"600 Block Commonwealth PL Pittsburgh, PA 15222",[5902(a)(1) Prostitution],"600 Block Commonwealth PL Pittsburgh, PA 15222",Central Business District,...,6,6,-80.006405,40.441733,0.057309,True,False,False,2018,3


### Q: Were any charges related to trafficking?

A: TODO
* 3011(b) Trafficking in Minors
* 3011(a)(1) Trafficking in Individuals Recruit/Entice etc 